# Autonomous Driving

## This project is based on the NVDIA's model for self driving cars. The model used for training is similar to that presented in their paper.

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split 
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
import argparse
import cv2,os
from keras.models import load_model
import matplotlib.image as mpimg
import ntpath

In [ ]:
np.random.seed(0)
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 160, 320, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)


## The images from the Udacity's self driving car simulator are loaded and preprocessed to provide the model with only the car postion on the road (without the sky and extra objects on the side)

In [ ]:
 def load_image(data_dir, image_file):
    #Change path based on where your data is located
    path = "drive/MyDrive/Autonomous/Data/IMG/"
    image_name = ntpath.basename(image_file)  
    image = path + image_name
    print(image)
    return mpimg.imread(image)

In [ ]:
def crop(image):
    return image[60:-25, :, :] # remove the sky and the car front

In [ ]:
def resize(image):
    return cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)

In [ ]:
def rgb2yuv(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2YUV)

In [ ]:
def preprocess(image):
    image = crop(image)
    image = resize(image)
    image = rgb2yuv(image)
    return image

In [ ]:
def choose_image(data_dir, center, left, right, steering_angle):
    choice = np.random.choice(3)
    if choice == 0:
        return load_image(data_dir, left), steering_angle + 0.2
    elif choice == 1:
        return load_image(data_dir, right), steering_angle - 0.2
    return load_image(data_dir, center), steering_angle

In [ ]:
def random_flip(image, steering_angle):
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        steering_angle = -steering_angle
    return image, steering_angle


In [ ]:
def random_translate(image, steering_angle, range_x, range_y):
    trans_x = range_x * (np.random.rand() - 0.5)
    trans_y = range_y * (np.random.rand() - 0.5)
    steering_angle += trans_x * 0.002
    trans_m = np.float32([[1, 0, trans_x], [0, 1, trans_y]])
    height, width = image.shape[:2]
    image = cv2.warpAffine(image, trans_m, (width, height))
    return image, steering_angle

In [ ]:
def random_shadow(image):
    x1, y1 = IMAGE_WIDTH * np.random.rand(), 0
    x2, y2 = IMAGE_WIDTH * np.random.rand(), IMAGE_HEIGHT
    xm, ym = np.mgrid[0:IMAGE_HEIGHT, 0:IMAGE_WIDTH]
    mask = np.zeros_like(image[:, :, 1])
    mask[np.where((ym - y1) * (x2 - x1) - (y2 - y1) * (xm - x1) > 0)] = 1
    cond = mask == np.random.randint(2)
    s_ratio = np.random.uniform(low=0.2, high=0.5)
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    hls[:, :, 1][cond] = hls[:, :, 1][cond] * s_ratio
    return cv2.cvtColor(hls, cv2.COLOR_HLS2RGB)

In [ ]:
def random_brightness(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    ratio = 1.0 + 0.4 * (np.random.rand() - 0.5)
    hsv[:,:,2] =  hsv[:,:,2] * ratio
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)


In [ ]:
def augument(data_dir, center, left, right, steering_angle, range_x=100, range_y=10):
    image, steering_angle = choose_image(data_dir, center, left, right, steering_angle)
    image, steering_angle = random_flip(image, steering_angle)
    image, steering_angle = random_translate(image, steering_angle, range_x, range_y)
    image = random_shadow(image)
    image = random_brightness(image)
    return image, steering_angle

In [ ]:
def batch_generator(data_dir, image_paths, steering_angles, batch_size, is_training):
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            steering_angle = steering_angles[index]
            # argumentation
            if is_training and np.random.rand() < 0.6:
                image, steering_angle = augument(data_dir, center, left, right, steering_angle)
            else:
                image = load_image(data_dir, center) 
            # add the image and steering angle to the batch
            images[i] = preprocess(image)
            steers[i] = steering_angle
            i += 1
            if i == batch_size:
                break
        yield images, steers


## Once the images are ready, the model can be built and trained. The layers, activation functions and number of units are based completely on the NVDIA model.

In [ ]:
def load_data(args):

    ## Change path based on where your data is located
    path = "drive/MyDrive/Autonomous/Data/"
    
    data_df = pd.read_csv(os.path.join(path + "driving_log.csv"), names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

    ## Only steering values are being predicted by this model. 

    X = data_df[['center', 'left', 'right']].values
    y = data_df['steering'].values
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=args['test_size'], random_state=0)

    return X_train, X_valid, y_train, y_valid

In [ ]:
def build_model(args):

    ##Same as the NVDIA model
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
    model.add(Conv2D(24, 5, 2, input_shape = (66, 200, 3), activation = 'relu'))
    model.add(Conv2D(36, 5, 2, activation = 'relu'))
    model.add(Conv2D(48, 5, 2, activation = 'relu'))
    model.add(Conv2D(64, 3, activation = 'relu'))
    model.add(Conv2D(64, 3, activation = 'relu'))
    model.add(Dropout(args['keep_prob']))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    model.summary()

    return model


In [ ]:
def train_model(model, args, X_train, X_valid, y_train, y_valid):
    ## Training based on the arguments
    checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=args['save_best_only'],
                                 mode='auto')
    model.compile(loss='mean_squared_error', optimizer=Adam(lr=args['learning_rate']))
    model.fit_generator(batch_generator(args['data_dir'], X_train, y_train, args['batch_size'], True),
                        args['samples_per_epoch'],
                        args['nb_epoch'],
                        max_queue_size=1,
                        validation_data=batch_generator(args['data_dir'], X_valid, y_valid, args['batch_size'], False),
                        validation_steps=len(X_valid),
                        callbacks=[checkpoint],
                        verbose=1)

In [ ]:

#The arguments can be changed according to one's requirement.
args = {'data_dir':'data', 'test_size':0.2, 'keep_prob':0.5, 'nb_epoch':25, 'samples_per_epoch':1000, 'batch_size':64, 'save_best_only':True, 'learning_rate': 0.0001}

#print parameters
print('-' * 30)
print('Parameters')
print(args)
print(args['data_dir'])
print('-' * 30)
print('-' * 30)

#load data
data = load_data(args)
#build model
model = build_model(args)
#train model on data, it saves as model.h5 
train_model(model, args, *data)

### The best model is saved after every epoch. This trained model can now be used to run a car on the Udacity's self driving car simulator 